# Comparison between patients of PD

In this notebook we will make a comparison for PD with early and late onset, taking into account motor and non motor symptoms.

In [ ]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os
import sys

path_to_lib = get_library_path()

sys.path.append(path_to_lib)
import luxgiant_clinical.TwoCatAnalysis as two
from luxgiant_clinical.Helpers import categories_recoder

import pandas as pd

In [ ]:
# load data

folder_path = os.path.join(path_to_lib, 'data/source')

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)

In [ ]:
# filter data to keep only patients with age at onset information

mask_patients= (df['Status']=='Patient')
mask_age     = (~df['agecat_1'].isnull()) 

df_cases = df[mask_patients & mask_age].reset_index(drop=True)
del df

df_cases.shape

In [ ]:
# rename and select first group of variables

variables_dict = {
    "initial_motor_symptom_s___1"   : "Onset with motor symptoms",
    "initial_motor_symptom_s___2"   : "Rest tremor as onset symptom" ,
    "initial_motor_symptom_s___12"  : "Bradykinesia as onset symptom" ,
    "initial_motor_symptom_s___13"  : "Rigidity as onset symptom",
    "initial_motor_symptom_s___3"   : "Postural/action tremor at onset",
    "initial_motor_symptom_s___8"   : "Reduced dexterity at onset",
    "initial_motor_symptom_s___9"   : "Microphagia at onset",
    "initial_motor_symptom_s___11"  : "Reduced arm swing at onset",
    "initial_motor_symptom_s___16"  : "Gait disorder at onset",
    "initial_motor_symptom_s___14"  : "Freezing at onset",
    "initial_motor_symptom_s___15"  : "Falls at onset",
    "initial_non_motor_symptom___1" : "Non Motor Symp at onset",
    "initial_non_motor_symptom___18": "Constipation at onset",
    "initial_non_motor_symptom___8" : "Insomnia as onset symptom",
    "initial_non_motor_symptom___4" : "RBD at onset",
    "initial_non_motor_symptom___4" : "RLS at onset",
    "initial_non_motor_symptom___20": "Urinary urgency as onset symptom",
    "initial_non_motor_symptom___13": "Hyperhidrosis as onset symptom",
    "initial_non_motor_symptom___10": "Anosmia/hyposmia as onset symptom",
    "initial_non_motor_symptom___2" : "Cognitive symptoms at onset",
    "current_motor_symptom_s___2"   : "Current rest tremor",
    "current_motor_symptom_s___3"   : "Current postural/action tremor",
    "current_motor_symptom_s___14"  : "Current bradykinesia",
    "current_motor_symptom_s___4"   : "Current rigidity",
    "current_motor_symptom_s___15"  : "Current Gait disorder",
    "current_motor_symptom_s___16"  : "Current Falls",
    "current_motor_symptom_s___17"  : "Current Freezing",
    "current_motor_symptom_s___18"  : "Current Dyskinesias",
    "current_motor_symptom_s___19"  : "Current Motor fluctuation",
    "current_non_motor_symptom___3" : "Current NMS Psychosis",
    "current_non_motor_symptom___5" : "Current depression",
    "current_non_motor_symptom___6" : "Current RBD",
    "current_non_motor_symptom___7" : "Current RLS",
    "current_non_motor_symptom___9" : "Current anosmia/hyposmia",
    "current_non_motor_symptom___2" : "Current cognitive impairment",
    "current_non_motor_symptom___10": "Current EDSS",
    "current_non_motor_symptom___11": "Current insomnia",
    "current_non_motor_symptom___14": "Current orthostasis",
    "current_non_motor_symptom___16": "Current hyperhidrosis",
    "current_non_motor_symptom___17": "Current seborrhea",
    "current_non_motor_symptom___20": "Current constipation",
    "current_non_motor_symptom___22": "Current urinary incontinence"
}
variables = ['participant_id', 'agecat_1'] + list(variables_dict.keys())

# statistical measures
stats_meas = {
    'n'     : ['Onset with motor symptoms', 'Rest tremor as onset symptom', 'Bradykinesia as onset symptom', 
               'Rigidity as onset symptom', 'Postural/action tremor at onset', 'Reduced dexterity at onset', 
               'Microphagia at onset', 'Reduced arm swing at onset', 'Gait disorder at onset', 'Freezing at onset', 
               'Falls at onset', 'Non Motor Symp at onset', 'Constipation at onset', 'Insomnia as onset symptom', 
               'RLS at onset', 'Urinary urgency as onset symptom', 'Hyperhidrosis as onset symptom', 
               'Anosmia/hyposmia as onset symptom', 'Cognitive symptoms at onset', 'Current rest tremor', 
               'Current postural/action tremor', 'Current bradykinesia', 'Current rigidity', 'Current Gait disorder', 
               'Current Falls', 'Current Freezing', 'Current Dyskinesias', 'Current Motor fluctuation', 
               'Current NMS Psychosis', 'Current depression', 'Current RBD', 'Current RLS', 
               'Current anosmia/hyposmia', 'Current cognitive impairment', 'Current EDSS', 'Current insomnia', 
               'Current orthostasis', 'Current hyperhidrosis', 'Current seborrhea', 'Current constipation', 
               'Current urinary incontinence']
    }

groups = ['Age at onset <50 years', 'Age at onset >=50 years']

In [ ]:
# keep only variables for analysis

df_cases_1 = df_cases[variables].copy()
df_cases_1 = df_cases_1.rename(columns=variables_dict)

del df_cases

In [ ]:
# recode variable values

motor_non_motor = ["Onset with motor symptoms", "Non Motor Symp at onset"]
remainig = [feat for feat in stats_meas['n'] if feat not in motor_non_motor]

df_cases_1 = categories_recoder(df_cases_1, motor_non_motor, {"Checked":0, "Unchecked":1})
df_cases_1 = categories_recoder(df_cases_1, remainig, {"Checked":1, "Unchecked":0})
df_cases_1 = categories_recoder(df_cases_1, ["agecat_1"], {"<50":'Age at onset <50 years', ">=50":'Age at onset >=50 years'})

In [ ]:
# count and proportion of categorical variables

df_6 = two.report_proportion(
    data_df    =df_cases_1,
    variables  =stats_meas['n'],
    groups     =groups,
    grouping_by='agecat_1',
)
df_6 = two.final_formatter(df_6, groups)

In [ ]:
# save result in csv file

df_6.to_csv(os.path.join(path_to_lib, 'data/auxiliar/aux_table_6.csv'), index=False)
df_6